In [1]:
from csgo.parser import DemoParser
import numpy
import csv
import pandas
import pyserialem
import os
from tabulate import tabulate

In [2]:
def getDemoPaths(dir_path):
    demoPaths = []
    x = 0
    for filename in os.listdir(dir_path):
        demoPaths.append(os.path.join(dir_path, filename))
    return demoPaths

In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    mapData, killData, nadeData, flashData, roundData, damageData = [], [], [], [], [], []
    for x in parsers:
        x.parse()
        mapData.append(x.parse(return_type="df"))
        killData.append(x._parse_kills(return_type="df"))
        nadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
    return (mapData, killData, nadeData, flashData, roundData, damageData)

In [5]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [6]:
def prettyPrintPlayerDamageTable(playerDamages, playerKills, playerNames, side, totalRounds):
    temp = extractDamageDifferentials(playerDamages, playerNames, side)
    temp1 = calculateKDByPlace(playerKills,playerNames, side)
    
    for key, val in temp1.items():
        if(key in temp.keys()):
            temp1[key]['Damage Dealt'] = temp[key]['Damage Dealt']
            temp1[key]['Damage Taken'] = temp[key]['Damage Taken']
            temp1[key]['Damage Differential'] = temp[key]['Damage Differential']
        else:
            temp1[key]['Damage Dealt'] = "N/A"
            temp1[key]['Damage Taken'] = "N/A"
            temp1[key]['Damage Differential'] = "N/A"
            
    prettyPrint = {"Places" : [], "Kills" : [], "Deaths" : [], "KD Diff" : [], "Damage Dealt" : [], "Damage Taken" : [], "Damage Differential" : [], "DD Per 30" : []}
    
    for key, val in temp1.items():
        prettyPrint['Places'].append(key)
        prettyPrint['Kills'].append(val['kills'])
        prettyPrint['Deaths'].append(val['deaths'])
        prettyPrint['KD Diff'].append(val['+-'])
        prettyPrint['Damage Dealt'].append(val['Damage Dealt'])
        prettyPrint['Damage Taken'].append(val['Damage Taken'])
        prettyPrint['Damage Differential'].append(val['Damage Differential'])
        if(val['Damage Differential'] != 'N/A'):
            prettyPrint['DD Per 30'].append(int(val['Damage Differential']) * 30 / totalRounds)
        else:
            prettyPrint['DD Per 30'].append('N/A')
    
    return(tabulate(prettyPrint, headers=prettyPrint.keys(), tablefmt="github"))

In [7]:
def extractDamageDifferentials(mapData, players, side):
    ret = {}
    for x in mapData[mapData['AttackerName'].isin(players)].AttackerAreaName.unique():
        damageDone = mapData[mapData['AttackerName'].isin(players)][mapData['AttackerSide'] == side][mapData['AttackerAreaName'] == x].HpDamageTaken.sum()
        damageTaken = mapData[mapData['VictimName'].isin(players)][mapData['VictimSide'] == side][mapData['VictimAreaName'] == x].HpDamageTaken.sum()
        ret[x] = {"Damage Dealt" : damageDone, "Damage Taken" : damageTaken, "Damage Differential" : (damageDone - damageTaken)}
    return ret

In [8]:
def calculateKDByPlace(kills, players, side):
    places = []
    temp = kills.copy()
    for x in temp.AttackerAreaName.unique():
        places.append(x)
    ret = {}
    for place in places:
        kill = len(kills[kills['AttackerName'].isin(players)][kills['AttackerAreaName'] == place][kills['AttackerSide'] == side])
        deaths = len(kills[kills['VictimName'].isin(players)][kills['VictimAreaName'] == place][kills['VictimSide'] == side])
        temp2 = {'kills' : kill, 'deaths' : deaths, '+-' : (kill-deaths)}
        ret[place] = temp2
    return ret

In [9]:
dir_path = "./demos/elige/inferno/Inferno"

In [10]:
demoPaths = getDemoPaths(dir_path)

In [11]:
demoParsers = generateDemoParsers(demoPaths)

16:50:52 [INFO] Go version>=1.14.0
16:50:52 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/astralis-vs-liquid-m1-inferno.dem
16:50:52 [INFO] Setting demo id to astralis-vs-liquid-m1-inferno
16:50:52 [INFO] Setting parse rate to 128
16:50:52 [INFO] Go version>=1.14.0
16:50:52 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/big-vs-liquid-m1-inferno.dem
16:50:52 [INFO] Setting demo id to big-vs-liquid-m1-inferno
16:50:52 [INFO] Setting parse rate to 128
16:50:52 [INFO] Go version>=1.14.0
16:50:52 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/endpoint-vs-liquid-m1-inferno_2346957.dem
16:50:52 [INFO] Setting demo id to endpoint-vs-liquid-m1-inferno
16:50:52 [INFO] Setting parse rate to 128
16:50:52 [INFO] Go version>=1.14.0
16:50:52 [INFO] Initialized CSGODemoParser with demofile /home/nk095x/cs_demos/csgo/demos/elige/inferno

# Demo Parsers: 16


In [12]:
print(len(demoPaths))

16


In [13]:
print(demoPaths)

['./demos/elige/inferno/Inferno/astralis-vs-liquid-m1-inferno.dem', './demos/elige/inferno/Inferno/big-vs-liquid-m1-inferno.dem', './demos/elige/inferno/Inferno/endpoint-vs-liquid-m1-inferno_2346957.dem', './demos/elige/inferno/Inferno/evil-geniuses-vs-liquid-m1-inferno.dem', './demos/elige/inferno/Inferno/evil-geniuses-vs-liquid-m1-inferno_2346960.dem', './demos/elige/inferno/Inferno/faze-vs-liquid-m2-inferno.dem', './demos/elige/inferno/Inferno/furia-vs-liquid-m2-inferno.dem', './demos/elige/inferno/Inferno/g2-vs-liquid-m3-inferno_2347019.dem', './demos/elige/inferno/Inferno/liquid-vs-astralis-m2-inferno.dem', './demos/elige/inferno/Inferno/liquid-vs-mousesports-m1-inferno.dem', './demos/elige/inferno/Inferno/liquid-vs-natus-vincere-m1-inferno.dem', './demos/elige/inferno/Inferno/liquid-vs-pain-m3-inferno.dem', './demos/elige/inferno/Inferno/liquid-vs-triumph-m1-inferno.dem', './demos/elige/inferno/Inferno/liquid-vs-triumph-m2-inferno_2347105.dem', './demos/elige/inferno/Inferno/mibr

In [14]:
maps, kills, nades, flashs, rounds, damages = generateData(demoParsers)

16:50:53 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:50:53 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/astralis-vs-liquid-m1-inferno.dem
16:51:11 [INFO] Wrote demo parse output to astralis-vs-liquid-m1-inferno.json
16:51:11 [INFO] Reading in JSON from astralis-vs-liquid-m1-inferno.json
16:51:12 [INFO] JSON data loaded, available in the `json` attribute to parser
16:51:12 [INFO] Successfully parsed JSON output
16:51:12 [INFO] Successfully returned JSON output
16:51:12 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:51:12 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/astralis-vs-liquid-m1-inferno.dem
16:51:30 [INFO] Wrote demo parse output to astralis-vs-liquid-m1-inferno.json
16:51:30 [INFO] Reading in JSON from astralis-vs-liquid-m1-inferno.json
16:51:31 [INFO] JSON data loaded, available in the `json` attribute to parser
16:51:31 [INFO] Successfully p

16:54:16 [INFO] Parsed rounds to Pandas DataFrame
16:54:16 [INFO] Parsed kills to Pandas DataFrame
16:54:16 [INFO] Parsed damages to Pandas DataFrame
16:54:16 [INFO] Parsed grenades to Pandas DataFrame
16:54:16 [INFO] Parsed flashes to Pandas DataFrame
16:54:16 [INFO] Parsed bomb_events to Pandas DataFrame
16:54:16 [INFO] Parsed frames to Pandas DataFrame
16:54:16 [INFO] Parsed player frames to Pandas DataFrame
16:54:16 [INFO] Returned dataframe output
16:54:16 [INFO] Parsed kills to Pandas DataFrame
16:54:16 [INFO] Parsed grenades to Pandas DataFrame
16:54:16 [INFO] Parsed flashes to Pandas DataFrame
16:54:16 [INFO] Parsed rounds to Pandas DataFrame
16:54:16 [INFO] Parsed damages to Pandas DataFrame
16:54:16 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:54:16 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/faze-vs-liquid-m2-inferno.dem
16:54:32 [INFO] Wrote demo parse output to faze-vs-liquid-m2-inferno.json
16:54:32 [IN

16:56:50 [INFO] Successfully parsed JSON output
16:56:50 [INFO] Successfully returned JSON output
16:56:50 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:56:50 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/liquid-vs-mousesports-m1-inferno.dem
16:57:11 [INFO] Wrote demo parse output to liquid-vs-mousesports-m1-inferno.json
16:57:11 [INFO] Reading in JSON from liquid-vs-mousesports-m1-inferno.json
16:57:11 [INFO] JSON data loaded, available in the `json` attribute to parser
16:57:11 [INFO] Successfully parsed JSON output
16:57:11 [INFO] Successfully returned JSON output
16:57:11 [INFO] Parsed rounds to Pandas DataFrame
16:57:11 [INFO] Parsed kills to Pandas DataFrame
16:57:11 [INFO] Parsed damages to Pandas DataFrame
16:57:11 [INFO] Parsed grenades to Pandas DataFrame
16:57:11 [INFO] Parsed flashes to Pandas DataFrame
16:57:11 [INFO] Parsed bomb_events to Pandas DataFrame
16:57:11 [INFO] Parsed frames to Pandas DataFrame
1

16:59:50 [INFO] Parsed kills to Pandas DataFrame
16:59:50 [INFO] Parsed grenades to Pandas DataFrame
16:59:50 [INFO] Parsed flashes to Pandas DataFrame
16:59:50 [INFO] Parsed rounds to Pandas DataFrame
16:59:50 [INFO] Parsed damages to Pandas DataFrame
16:59:50 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
16:59:50 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/mibr-vs-liquid-m2-inferno.dem
17:00:07 [INFO] Wrote demo parse output to mibr-vs-liquid-m2-inferno.json
17:00:07 [INFO] Reading in JSON from mibr-vs-liquid-m2-inferno.json
17:00:08 [INFO] JSON data loaded, available in the `json` attribute to parser
17:00:08 [INFO] Successfully parsed JSON output
17:00:08 [INFO] Successfully returned JSON output
17:00:08 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
17:00:08 [INFO] Looking for file at /home/nk095x/cs_demos/csgo/demos/elige/inferno/Inferno/mibr-vs-liquid-m2-inferno.dem
17:00:25 [INFO] Wrote

In [15]:
failedDemos = []
eligeMaps, eligeKills, eligeNades, eligeFlashs, eligeRounds, eligeDamages = [], [], [], [], [], []
for x in range(len(kills)):
    if(kills[x].shape[0] > 100):
        eligeMaps.append(maps[x])
        eligeKills.append(kills[x])
        eligeNades.append(nades[x])
        eligeFlashs.append(flashs[x])
        eligeRounds.append(rounds[x])
        eligeDamages.append(damages[x])
    else:
        failedDemos.append(x)

In [16]:
killData = concat_data(eligeKills)
nadeData = concat_data(eligeNades)
flashData = concat_data(eligeFlashs)
roundData = concat_data(eligeRounds)
damageData = concat_data(eligeDamages)

In [17]:
rounds = 0
for x in roundData.MatchId.unique():
    rounds += roundData[roundData['MatchId'] == x].RoundNum.iloc[-1]
rounds

387

In [18]:
flashData.columns

Index(['Tick', 'Second', 'AttackerSteamId', 'AttackerName', 'AttackerTeam',
       'AttackerSide', 'AttackerX', 'AttackerY', 'AttackerZ', 'AttackerAreaId',
       'AttackerAreaName', 'AttackerViewX', 'AttackerViewY', 'PlayerSteamId',
       'PlayerName', 'PlayerTeam', 'PlayerSide', 'PlayerX', 'PlayerY',
       'PlayerZ', 'PlayerAreaId', 'PlayerAreaName', 'PlayerViewX',
       'PlayerViewY', 'RoundNum', 'MatchId', 'MapName'],
      dtype='object')

In [19]:
eligeDamage = damageData[(damageData['AttackerName'] == 'EliGE') |(damageData['VictimName'] == 'EliGE')]

In [20]:
eligeKills = killData[(killData['AttackerName'] == 'EliGE') | (killData['VictimName'] == 'EliGE')]

In [21]:
## Frequency Distribution of Weapons Used when getting Kills
eligeKills[eligeKills['AttackerName'] == 'EliGE'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

AK-47                 44.5%
M4A4                  15.2%
AUG                   14.8%
Desert Eagle           8.1%
Glock-18               3.9%
USP-S                  3.5%
FAMAS                  2.1%
Galil AR               1.4%
MP9                    1.1%
Molotov                1.1%
UMP-45                 1.1%
P2000                  0.7%
MAC-10                 0.7%
HE Grenade             0.7%
p250                   0.4%
Incendiary Grenade     0.4%
Dual Berettas          0.4%
Name: Weapon, dtype: object

In [22]:
## Frequency Distribution of Weapons Used when getting Kills on CT Side
eligeKills[eligeKills['AttackerName'] == 'EliGE'][eligeKills['AttackerSide'] == 'CT'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AUG                   27.3%
M4A4                  25.3%
AK-47                 20.8%
Desert Eagle          11.0%
USP-S                  6.5%
FAMAS                  3.9%
P2000                  1.3%
UMP-45                 0.6%
MP9                    0.6%
p250                   0.6%
Incendiary Grenade     0.6%
Dual Berettas          0.6%
HE Grenade             0.6%
Name: Weapon, dtype: object

In [23]:
## Frequency Distribution of Weapons Used when getting Kills on T Side
eligeKills[eligeKills['AttackerName'] == 'EliGE'][eligeKills['AttackerSide'] == 'T'].Weapon.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


AK-47           72.9%
Glock-18         8.5%
Desert Eagle     4.7%
M4A4             3.1%
Galil AR         3.1%
Molotov          2.3%
MP9              1.6%
MAC-10           1.6%
UMP-45           1.6%
HE Grenade       0.8%
Name: Weapon, dtype: object

In [24]:
## Frequency Distribution of your location when you got a kill
eligeKills[eligeKills['AttackerName'] == 'EliGE'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

BombsiteA     20.8%
BombsiteB     17.0%
Apartments    11.3%
Banana         9.9%
TopofMid       8.8%
Middle         6.4%
Pit            5.7%
Graveyard      3.9%
Balcony        3.2%
CTSpawn        3.2%
Ruins          3.2%
Arch           2.8%
Library        1.8%
Quad           0.7%
LowerMid       0.4%
Underpass      0.4%
SecondMid      0.4%
TRamp          0.4%
Name: AttackerAreaName, dtype: object

In [25]:
## Frequency Distribution of your location when you got a kill on CT Side
eligeKills[eligeKills['AttackerName'] == 'EliGE'][eligeKills['AttackerSide'] == 'CT'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BombsiteA     27.9%
BombsiteB     13.6%
TopofMid       9.7%
Graveyard      7.1%
Apartments     7.1%
Pit            5.8%
Banana         5.2%
Ruins          5.2%
CTSpawn        3.9%
Middle         3.2%
Arch           3.2%
Balcony        2.6%
Library        2.6%
Quad           1.3%
TRamp          0.6%
LowerMid       0.6%
Name: AttackerAreaName, dtype: object

In [26]:
## Frequency Distribution of your location when you got a kill on T Side
eligeKills[eligeKills['AttackerName'] == 'EliGE'][eligeKills['AttackerSide'] == 'T'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BombsiteB     20.9%
Apartments    16.3%
Banana        15.5%
BombsiteA     12.4%
Middle        10.1%
TopofMid       7.8%
Pit            5.4%
Balcony        3.9%
Arch           2.3%
CTSpawn        2.3%
Ruins          0.8%
SecondMid      0.8%
Library        0.8%
Underpass      0.8%
Name: AttackerAreaName, dtype: object

In [27]:
## Frequency Distribution of your location when you got a kill on CT Side and its the first kill in the round (5v4 creation opportunities)
eligeKills[eligeKills['AttackerName'] == 'EliGE'][eligeKills['AttackerSide'] == 'CT'][eligeKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BombsiteA     27.6%
Apartments    27.6%
TopofMid      20.7%
Balcony        3.4%
LowerMid       3.4%
Graveyard      3.4%
BombsiteB      3.4%
Library        3.4%
Banana         3.4%
Arch           3.4%
Name: AttackerAreaName, dtype: object

In [28]:
## Frequency Distribution of your location when you got a kill on T Side and its the first kill in the round (5v4 creation opportunities)
eligeKills[eligeKills['AttackerName'] == 'EliGE'][eligeKills['AttackerSide'] == 'T'][eligeKills['IsFirstKill'] == True].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Apartments    33.3%
Middle        22.2%
Banana        16.7%
Arch           5.6%
Balcony        5.6%
BombsiteA      5.6%
BombsiteB      5.6%
SecondMid      5.6%
Name: AttackerAreaName, dtype: object

In [29]:
## Printed table of volume kills, deaths, kd differentials, volume damage dealt, damage taken, and damage diferential also damage differnetial scaled over 30 rounds, all these aforementioned metrics broken down by locaiton. THis is for CT Side
print(prettyPrintPlayerDamageTable(eligeDamage, eligeKills, ['EliGE'], 'CT', rounds))

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


| Places     |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   | DD Per 30           |
|------------|---------|----------|-----------|----------------|----------------|-----------------------|---------------------|
| Banana     |       8 |        2 |         6 | 441            | 249            | 192                   | 14.883720930232558  |
| BombsiteB  |      21 |       22 |        -1 | 1928           | 2513           | -585                  | -45.348837209302324 |
| Apartments |      11 |       15 |        -4 | 1349           | 1763           | -414                  | -32.093023255813954 |
| Graveyard  |      11 |        6 |         5 | 1074           | 530            | 544                   | 42.17054263565891   |
| BackAlley  |       0 |        1 |        -1 | N/A            | N/A            | N/A                   | N/A                 |
| BombsiteA  |      43 |       30 |        13 | 4310           | 3627           | 683                   

In [30]:
## Printed table of volume kills, deaths, kd differentials, volume damage dealt, damage taken, and damage diferential also damage differnetial scaled over 30 rounds, all these aforementioned metrics broken down by locaiton. THis is for T Side
print(prettyPrintPlayerDamageTable(eligeDamage, eligeKills, ['EliGE'], 'T', rounds))

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


| Places     |   Kills |   Deaths |   KD Diff | Damage Dealt   | Damage Taken   | Damage Differential   | DD Per 30            |
|------------|---------|----------|-----------|----------------|----------------|-----------------------|----------------------|
| Banana     |      20 |       17 |         3 | 1919           | 2044           | -125                  | -9.689922480620154   |
| BombsiteB  |      27 |       33 |        -6 | 2875           | 3307           | -432                  | -33.48837209302326   |
| Apartments |      21 |       14 |         7 | 2215           | 1803           | 412                   | 31.93798449612403    |
| Graveyard  |       0 |        1 |        -1 | 189            | 125            | 64                    | 4.961240310077519    |
| BackAlley  |       0 |        0 |         0 | N/A            | N/A            | N/A                   | N/A                  |
| BombsiteA  |      16 |       18 |        -2 | 1905           | 1962           | -57            

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [31]:
#frequency distribution of the location your opponent was at when they flashed you as a ct
flashData[flashData['PlayerName'] == 'EliGE'][flashData['PlayerSide'] == 'CT'][flashData['AttackerTeam'] != 'Team Liquid'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Middle        27.3%
Banana        19.6%
Apartments    13.3%
BombsiteA      9.1%
TopofMid       7.0%
SecondMid      7.0%
TRamp          3.5%
BombsiteB      2.8%
Balcony        2.8%
Ruins          1.4%
Quad           1.4%
Graveyard      1.4%
Bridge         0.7%
Arch           0.7%
LowerMid       0.7%
TSpawn         0.7%
Underpass      0.7%
Name: AttackerAreaName, dtype: object

In [32]:
## Va
flashData[flashData['PlayerName'] == 'EliGE'][flashData['AttackerTeam'] != 'Team Liquid'].AttackerTeam.value_counts()

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


Astralis         41
Evil Geniuses    34
Triumph          33
G2 Esports       30
Natus Vincere    26
FaZe Clan        21
BIG              20
paiN Gaming      16
mousesports      14
MIBR             14
Endpoint CeX     10
FURIA             8
Name: AttackerTeam, dtype: int64

In [33]:
#frequency distribution of the location you were at when your opponent flashed you. you are a ct.
flashData[flashData['PlayerName'] == 'EliGE'][flashData['PlayerSide'] == 'CT'][flashData['AttackerTeam'] != 'Team Liquid'].PlayerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BombsiteA     22.4%
TopofMid      21.0%
Pit            8.4%
Middle         7.0%
Graveyard      6.3%
Apartments     6.3%
BombsiteB      5.6%
Balcony        4.9%
Ruins          4.9%
Banana         4.2%
Quad           2.8%
Library        2.1%
Arch           1.4%
CTSpawn        1.4%
TRamp          0.7%
BackAlley      0.7%
Name: PlayerAreaName, dtype: object

In [34]:
#frequency distribution of the location your opponent was at when they flashed you. You are a T
flashData[flashData['PlayerName'] == 'EliGE'][flashData['PlayerSide'] == 'T'][flashData['AttackerTeam'] != 'Team Liquid'].AttackerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BombsiteA     24.2%
BombsiteB     20.2%
Ruins         13.7%
CTSpawn       12.1%
TopofMid       8.9%
Arch           4.8%
Banana         4.8%
Pit            4.0%
Middle         2.4%
Balcony        1.6%
Apartments     1.6%
Library        1.6%
Name: AttackerAreaName, dtype: object

In [35]:
#frequency distribution of the location you were at when your opponent flashed you. you are a t.
flashData[flashData['PlayerName'] == 'EliGE'][flashData['PlayerSide'] == 'T'][flashData['AttackerTeam'] != 'Team Liquid'].PlayerAreaName.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'

/home/nk095x/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


BombsiteB     19.4%
Banana        18.5%
TopofMid      16.1%
Apartments    10.5%
BombsiteA      8.1%
SecondMid      8.1%
Middle         4.8%
Pit            4.0%
Underpass      2.4%
Arch           1.6%
Balcony        1.6%
Quad           1.6%
Graveyard      1.6%
Kitchen        0.8%
TSpawn         0.8%
Name: PlayerAreaName, dtype: object